In [54]:
# INGESTÃO
# Parâmetros de Conexão

from sqlalchemy import create_engine

# Dados da conexão
usuario = "root"                 # usuário do banco de dados
senha = "Servidor2011"           # senha
host = "127.0.0.1"               # IP endereço do banco
porta = "3306"                   # porta padrão do MySQL
banco = "dw_ep_producao"         # nome do seu schema ou banco

# Conexão com o banco de dados MySQL
engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}")


In [55]:
# Teste de conexão e visualização de tabelas

import pandas as pd

try:
    df = pd.read_sql("SHOW TABLES;", engine)
    print("Conexão bem-sucedida. Tabelas disponíveis:")
    print(df)
except Exception as e:
    print(f"Erro na conexão: {e}")

Conexão bem-sucedida. Tabelas disponíveis:
  Tables_in_dw_ep_producao
0              dim_produto
1                dim_tempo
2              dim_unidade
3               dim_versao
4            fato_producao


In [56]:
# Configuração de logging

import os
import logging

# Garante que o diretório de logs exista
os.makedirs("logs", exist_ok=True)

# Configuração do log
logging.basicConfig(filename='logs/pre-processamento.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [57]:
# Execução da query e padronização

query = """
SELECT data_producao, volume_m3

FROM dw_ep_producao.fato_producao;
"""


In [58]:
# Limpeza e ordenação
# Essencial para garantir integridade da série temporal

try:
    # Carregar os dados em um DataFrame
    df = pd.read_sql(query, engine)
    
    # Limpeza e organização
    df = df.drop_duplicates()  # Remove duplicatas
    df['data_producao'] = pd.to_datetime(df['data_producao'])  # Converte para datetime
    df = df.sort_values('data_producao')  # Ordena por data

    # Exibir o DataFrame final
    print(df)

except Exception as e:
    print(f"Ocorreu um erro: {e}")

    data_producao  volume_m3
0      2024-01-01    139.214
1      2024-01-02      0.000
2      2024-01-03    102.821
3      2024-01-04     76.128
4      2024-01-05     89.816
..            ...        ...
361    2024-12-27      0.000
362    2024-12-28      0.000
363    2024-12-29      0.000
364    2024-12-30      0.000
365    2024-12-31      0.000

[366 rows x 2 columns]


In [59]:
pd.read_sql("DESCRIBE dw_ep_producao.fato_producao;", con=engine)


,Field,Type,Null,Key,Default,Extra
0,id_fato_producao,int,NO,PRI,None,auto_increment
1,data_producao,date,NO,,None,
2,unidade_id,int,YES,MUL,None,
3,produto_id,int,YES,MUL,None,
4,versao_id,int,YES,MUL,None,
5,volume_m3,double,YES,,None,
6,tempo_id,int,YES,MUL,None,


In [60]:
df.to_csv("C:/Users/RODRIGUES/Desktop/ANALISE_EXPLORATORIO/PIPELINE/Pipeline_de_Tendências_Temporais/dados_padronizados2.csv", index=False)
